In [1]:
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K

from keras.callbacks import TensorBoard
# dimensions of our images. 
img_width, img_height = 150, 150

train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/validation'

nb_train_samples = 2388
nb_validation_samples = 600
epochs = 12
batch_size = 249

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 
    
model = Sequential() 
model.add(Conv2D(32, (3, 3), input_shape=input_shape)) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(64, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(5)) #5分类 
model.add(Activation('softmax')) #采用Softmax 

model.summary()

model.compile(loss='categorical_crossentropy', #多分类 
              optimizer='rmsprop', 
              metrics=['accuracy']) 

tbCallBack = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

# this is the augmentation configuration we will use for training 
train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True) 

# this is the augmentation configuration we will use for testing: 
# only rescaling 
test_datagen = ImageDataGenerator(rescale=1. / 255) 

train_generator = train_datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='categorical') #多分类 

validation_generator = test_datagen.flow_from_directory( 
    validation_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='categorical') #多分类 

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

//anaconda3/lib/python3.7/site-packages/keras/callbacks/tensorboard_v2.py:97: UserWarning: The TensorBoard callback does not support gradients display when using TensorFlow 2.0. The `write_grads` argument is ignored.
  warnings.warn('The TensorBoard callback does not support '


Found 2382 images belonging to 5 classes.
Found 594 images belonging to 5 classes.


In [2]:
from time import time
from tensorflow.python.keras.callbacks import TensorBoard

model.fit_generator( 
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size, 
    epochs=epochs, 
    callbacks=[tbCallBack],
    validation_data=validation_generator, 
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/12
9/9 [==============================] - 157s 17s/step - loss: 2.4818 - accuracy: 0.6033 - val_loss: 0.8725 - val_accuracy: 0.6446
Epoch 2/12
9/9 [==============================] - 128s 14s/step - loss: 0.8278 - accuracy: 0.7032 - val_loss: 0.8599 - val_accuracy: 0.7420
Epoch 3/12
9/9 [==============================] - 146s 16s/step - loss: 0.5671 - accuracy: 0.8211 - val_loss: 0.6182 - val_accuracy: 0.8986
Epoch 4/12
9/9 [==============================] - 131s 15s/step - loss: 0.3067 - accuracy: 0.9077 - val_loss: 0.3706 - val_accuracy: 0.8795
Epoch 5/12
9/9 [==============================] - 157s 17s/step - loss: 0.5170 - accuracy: 0.8411 - val_loss: 0.0668 - val_accuracy: 1.0000
Epoch 6/12
9/9 [==============================] - 128s 14s/step - loss: 0.1648 - accuracy: 0.9527 - val_loss: 0.1501 - val_accuracy: 0.9855
Epoch 7/12
9/9 [==============================] - 128s 14s/step - loss: 0.1405 - accuracy: 0.9522 - val_loss: 0.0087 - val_accuracy: 0.9980
Epoch 8/12
9/9 [====

In [ ]:
#tensorboard --logdir ./logs 

In [4]:
model.save('All.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image
img = 'test/IMG_2922.jpg'

img = image.load_img(img, target_size=(150,150))
img.show()

x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

result = model.predict(x)
print(result)

Max = 0.0
for j in result:
    for i, c in enumerate(j):
        if c > Max:
            print(c, Max)
            Max = i
            a = i
        
print(a)

if a == 0:
    print('addidas')
    
elif a == 1:
    print('pierre cardin')
    
elif a == 2:
    print('春水堂')
    
elif a == 3:
    print('幸福台灣')
    
elif a == 4:
    print('Timberland')
    
else:
    raise IndexError